In [1]:
# Standard Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
# Additional Imports
import os, json, math, time
from yelpapi import YelpAPI
from tqdm.notebook import tqdm_notebook

In [2]:
# Load API Credentials
with open('/Users/Admin/.secret/yelp_api.json') as f:   #use your path here!
    login = json.load(f)
login.keys()

dict_keys(['Client-ID', 'API Key'])

In [3]:
# Instantiate YelpAPI Variable
yelp_api = YelpAPI(login['API Key'], timeout_s=5.0)

In [4]:
# set API call parameters 
LOCATION = 'Frisco, TX'
TERM = 'Tacos'


In [7]:
# Specifying JSON_FILE filename
JSON_FILE = "Data/results_in_progress_TX_Tacos.json"
JSON_FILE

'Data/results_in_progress_TX_Tacos.json'

In [10]:
def create_json_file(JSON_FILE,  delete_if_exists=False):
    
    ## Check if JSON_FILE exists
    file_exists = os.path.isfile(JSON_FILE)
    
    ## If it DOES exist:
    if file_exists == True:
        
        ## Check if user wants to delete if exists
        if delete_if_exists==True:
            
            print(f"[!] {JSON_FILE} already exists. Deleting previous file...")
            ## delete file and confirm it no longer exits.
            os.remove(JSON_FILE)
            ## Recursive call to function after old file deleted
            create_json_file(JSON_FILE,delete_if_exists=False)
        else:
            print(f"[i] {JSON_FILE} already exists.")            
            
            
    ## If it does NOT exist:
    else:
        
        ## INFORM USER AND SAVE EMPTY LIST
        print(f"[i] {JSON_FILE} not found. Saving empty list to new file.")
        
        ## CREATE ANY NEEDED FOLDERS
        # Get the Folder Name only
        folder = os.path.dirname(JSON_FILE)
        
        ## If JSON_FILE included a folder:
        if len(folder)>0:
            # create the folder
            os.makedirs(folder,exist_ok=True)
        ## Save empty list to start the json file
        with open(JSON_FILE,'w') as f:
            json.dump([],f)

In [11]:
## Create a new empty json file (exist the previous if it exists)
create_json_file(JSON_FILE, delete_if_exists=True)
## Load previous results and use len of results for offset
with open(JSON_FILE,'r') as f:
    previous_results = json.load(f)
    
## set offset based on previous results
n_results = len(previous_results)
print(f'- {n_results} previous results found.')
# use our yelp_api variable's search_query method to perform our API call
results = yelp_api.search_query(location=LOCATION,
                                term=TERM,
                               offset=n_results)
## How many results total?
total_results = results['total']
## How many did we get the details for?
results_per_page = len(results['businesses'])
# Use math.ceil to round up for the total number of pages of results.
n_pages = math.ceil((results['total']-n_results)/ results_per_page)
n_pages

[i] Data/results_in_progress_TX_Tacos.json not found. Saving empty list to new file.
- 0 previous results found.


55

In [12]:
for i in tqdm_notebook( range(1,n_pages+1)):
    
    ## Read in results in progress file and check the length
    with open(JSON_FILE, 'r') as f:
        previous_results = json.load(f)
    ## save number of results for to use as offset
    n_results = len(previous_results)
    
    if (n_results + results_per_page) > 1000:
        print('Exceeded 1000 api calls. Stopping loop.')
        break
    
    ## use n_results as the OFFSET 
    results = yelp_api.search_query(location=LOCATION,
                                    term=TERM, 
                                    offset=n_results)
    
    
    
    ## append new results and save to file
    previous_results.extend(results['businesses'])
    
    # display(previous_results)
    with open(JSON_FILE,'w') as f:
        json.dump(previous_results,f)
    
    time.sleep(.2)

  0%|          | 0/55 [00:00<?, ?it/s]

Exceeded 1000 api calls. Stopping loop.


In [13]:
# load final results
final_df = pd.read_json(JSON_FILE)
display(final_df.head(), final_df.tail())

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
0,opFjbZ9NAOjefao_zGly6Q,chído-taco-lounge-frisco,chído Taco Lounge,https://s3-media3.fl.yelpcdn.com/bphoto/GdGCON...,False,https://www.yelp.com/biz/ch%C3%ADdo-taco-loung...,195,"[{'alias': 'tacos', 'title': 'Tacos'}]",4.5,"{'latitude': 33.12186180097871, 'longitude': -...","[delivery, pickup]",$$,"{'address1': '6959 Lebanon Rd', 'address2': No...",+14692005715,(469) 200-5715,1246.114153
1,A3HeeuhyfPJsM04YPzsY5w,marianas-taco-shop-frisco,Marianas Taco Shop,https://s3-media3.fl.yelpcdn.com/bphoto/4or9qQ...,False,https://www.yelp.com/biz/marianas-taco-shop-fr...,468,"[{'alias': 'mexican', 'title': 'Mexican'}]",4.0,"{'latitude': 33.151359245193, 'longitude': -96...",[delivery],$,"{'address1': '8981 5th St', 'address2': '', 'a...",+12147059059,(214) 705-9059,2162.011494
2,LyTOhB3pINm9npfX2g2dcA,frezko-taco-spot-the-colony,Frezko Taco Spot,https://s3-media2.fl.yelpcdn.com/bphoto/pzKafQ...,False,https://www.yelp.com/biz/frezko-taco-spot-the-...,829,"[{'alias': 'mexican', 'title': 'Mexican'}]",4.5,"{'latitude': 33.070126, 'longitude': -96.881802}","[delivery, pickup]",$$,"{'address1': '5101 TX-121', 'address2': 'Suite...",+19722945620,(972) 294-5620,9058.175382
3,J6-PwOfRohS4gGriZciTVw,la-suprema-market-frisco,La Suprema Market,https://s3-media3.fl.yelpcdn.com/bphoto/u49t5t...,False,https://www.yelp.com/biz/la-suprema-market-fri...,89,"[{'alias': 'intlgrocery', 'title': 'Internatio...",4.5,"{'latitude': 33.151254, 'longitude': -96.82714}",[delivery],$,"{'address1': '6726 Main St', 'address2': 'Ste ...",+14692940072,(469) 294-0072,2233.754368
4,Yfd_q1H6EHs8Jcus0YzEWw,taco-ocho-frisco,Taco Ocho,https://s3-media2.fl.yelpcdn.com/bphoto/Rh4voL...,False,https://www.yelp.com/biz/taco-ocho-frisco?adju...,270,"[{'alias': 'mexican', 'title': 'Mexican'}, {'a...",4.0,"{'latitude': 33.1077123837669, 'longitude': -9...","[delivery, pickup]",$$,"{'address1': '3492 Legacy Dr', 'address2': 'St...",+12144942346,(214) 494-2346,3374.443061


,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
995,oXuxRsNrLXx7QmH2MRbBRg,wendys-frisco-4,Wendy's,https://s3-media1.fl.yelpcdn.com/bphoto/LmWMjG...,False,https://www.yelp.com/biz/wendys-frisco-4?adjus...,53,"[{'alias': 'hotdogs', 'title': 'Fast Food'}, {...",1.5,"{'latitude': 33.174393, 'longitude': -96.837981}",[delivery],$,"{'address1': '5555 El Dorado Parkway', 'addres...",+12147058392,(214) 705-8392,4998.372164
996,2w3OLEu0BTG-DvBgONMcqQ,whole-foods-market-richardson-5,Whole Foods Market,https://s3-media2.fl.yelpcdn.com/bphoto/3omwgN...,False,https://www.yelp.com/biz/whole-foods-market-ri...,126,"[{'alias': 'organic_stores', 'title': 'Organic...",3.5,"{'latitude': 32.99871547899343, 'longitude': -...",[],$$,"{'address1': '1411 E Renner Rd', 'address2': '...",+12142730902,(214) 273-0902,18612.666229
997,-yB7Y3vXdtmjJkErBXn23A,7-eleven-the-colony,7-Eleven,https://s3-media2.fl.yelpcdn.com/bphoto/kmCZcV...,False,https://www.yelp.com/biz/7-eleven-the-colony?a...,6,"[{'alias': 'coffee', 'title': 'Coffee & Tea'},...",1.0,"{'latitude': 33.0588716, 'longitude': -96.8624...","[delivery, pickup]",$,"{'address1': '5801 Windhaven Pkwy', 'address2'...",+12144691665,(214) 469-1665,9112.417703
998,Fpcsb3ylccCaQ8-oqugrLA,southern-recipes-grill-plano,Southern Recipes Grill,https://s3-media2.fl.yelpcdn.com/bphoto/DzDzea...,False,https://www.yelp.com/biz/southern-recipes-gril...,88,"[{'alias': 'tradamerican', 'title': 'American ...",3.0,"{'latitude': 33.010061, 'longitude': -96.712225}","[delivery, pickup]",$,"{'address1': '621 W Plano Pkwy', 'address2': '...",+19723129800,(972) 312-9800,16849.591240
999,ZNXdqXkZDc5CLWsGdHLsUg,ramen-hakata-lewisville,Ramen Hakata,https://s3-media3.fl.yelpcdn.com/bphoto/uz9UaK...,False,https://www.yelp.com/biz/ramen-hakata-lewisvil...,460,"[{'alias': 'ramen', 'title': 'Ramen'}]",4.0,"{'latitude': 33.002834823772126, 'longitude': ...","[delivery, pickup]",$$,"{'address1': '500 E Round Grove Rd', 'address2...",+14697639006,(469) 763-9006,20600.853425


In [14]:
# save the final results to a compressed csv
final_df.to_csv('Data/final_results_TX_Tacos.csv.gz', compression='gzip',index=False)